## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

        - Nombre: Matías
- Apellido: Schettino
- Legajo: 1635244

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
dataset = pd.read_csv(r'C:\Users\matis\Documents\Facultad\Ciencia de datos\TP2_airbnb_london.csv', delimiter=',',encoding='latin-1')

In [4]:
#muesto las primeras columnas del dataset

dataset.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [5]:
print("El dataset  tiene "+ str(np.shape(dataset)[0]) + " renglones y " + str(np.shape(dataset)[1]) + " columnas")

El dataset  tiene 51827 renglones y 9 columnas


In [6]:
del(dataset['Unnamed: 0']) 


In [7]:
dataset.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [8]:
col_nulls = dataset.isnull().any()
col_nulls

neighbourhood                     False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [9]:
  dataset.dtypes

neighbourhood                      object
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [17]:


## corrí primero la division en train y test para estandarizar y despues hice las dummies.
neigh_dummies = pd.get_dummies(dataset['neighbourhood'], drop_first = True)
neigh_dummies.head()

,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,Greenwich,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Imprimir las dimensiones y primeras lineas de estas dummies

In [18]:
neigh_dummies.head()

,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,Greenwich,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(dataset['room_type'], drop_first = True)
room_dummies.head()

,Private room,Shared room
0,1,0
1,1,0
2,1,0
3,0,0
4,1,0


Imprimir las dimensiones y primeras lineas de estas dummies

Joinear las variables dummies al dataset



In [13]:
london = dataset.join(room_dummies)
london2 = london.join(neigh_dummies)
london2.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Private room,Shared room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,1,0,...,0,0,0,0,0,0,0,0,0,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [12]:

y = np.array(dataset[["price"]])
x = dataset.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [14]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [15]:
xtrain_scal = scaler.transform(xtrain)

In [16]:
xtest_scal = scaler.transform(xtest)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [20]:
def lin_reg(x,y):
  n, m = x.shape
  I = np.identity(m)
  w = np.dot(np.dot(np.linalg.inv(np.dot(x.T, x)), x.T), y)
  return w

In [21]:
weights_ols = lin_reg(xtrain,ytrain)
weights_ols

array([[19.99110903],
       [-0.07492092],
       [ 9.45727787],
       [ 0.16329495],
       [ 0.12199513]])

In [22]:
y_pred_val_ols = np.dot(xtest, np.asarray(weights_ols)) #Aca hago la predicción

In [23]:
rmse_ols = np.sqrt(np.mean(np.square(ytest - y_pred_val_ols)))
rmse_ols
#Error cuadratico medio

61.669537528108904

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [27]:
svreg = SVR()
param_svreg = {'kernel':('linear', 'rbf'), 'C':[1, 2,], 'gamma':[ 0.1,1,]} #no uso tres parametros xq me tardo 3 horas en correr todo.

In [28]:
regressor_svr = GridSearchCV(svreg, param_svreg, cv=5, verbose=3)

In [30]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.007 total time=  28.2s
[CV 2/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.005 total time=  28.0s
[CV 3/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.033 total time=  27.8s
[CV 4/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.017 total time=  28.4s
[CV 5/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.015 total time=  28.7s
[CV 1/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.068 total time=  44.0s
[CV 2/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.069 total time=  43.8s
[CV 3/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.087 total time=  43.8s
[CV 4/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.082 total time=  44.2s
[CV 5/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.082 total time=  44.8s
[CV 1/5] END .......C=1, gamma=1, kernel=linear;, score=0.007 total time=  27.9s
[CV 2/5] END .......C=1, gamma=1, kernel=linear;,

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 2], 'gamma': [0.1, 1],
                         'kernel': ('linear', 'rbf')},
             verbose=3)

In [31]:
regressor_svr.best_params_

{'C': 2, 'gamma': 1, 'kernel': 'rbf'}

In [32]:
regressor_svr.best_score_

0.0955961032479145

In [33]:
ypred3 = regressor_svr.predict(xtest_scal)
ypred3

array([86.98104105, 78.95338906, 81.35958711, ..., 89.31584041,
       71.2749561 , 70.20255105])

In [34]:
np.sqrt(mean_squared_error(ytest, ypred3))

52.976767911191246

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [36]:
svreg2 = SVR()
param_svreg2 = {'kernel':('linear', 'rbf'), 'C':[10, 4], 'gamma':[ 0.5,5]}

In [37]:
regressor_svr2 = GridSearchCV(svreg2, param_svreg2, cv=5, verbose=3)

In [38]:
regressor_svr2.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ....C=10, gamma=0.5, kernel=linear;, score=0.007 total time=  31.1s
[CV 2/5] END ....C=10, gamma=0.5, kernel=linear;, score=0.005 total time=  31.1s
[CV 3/5] END ....C=10, gamma=0.5, kernel=linear;, score=0.033 total time=  30.9s
[CV 4/5] END ....C=10, gamma=0.5, kernel=linear;, score=0.016 total time=  32.3s
[CV 5/5] END ....C=10, gamma=0.5, kernel=linear;, score=0.015 total time=  33.6s
[CV 1/5] END .......C=10, gamma=0.5, kernel=rbf;, score=0.103 total time=  44.6s
[CV 2/5] END .......C=10, gamma=0.5, kernel=rbf;, score=0.099 total time=  45.4s
[CV 3/5] END .......C=10, gamma=0.5, kernel=rbf;, score=0.121 total time=  45.8s
[CV 4/5] END .......C=10, gamma=0.5, kernel=rbf;, score=0.118 total time=  44.9s
[CV 5/5] END .......C=10, gamma=0.5, kernel=rbf;, score=0.109 total time=  45.0s
[CV 1/5] END ......C=10, gamma=5, kernel=linear;, score=0.007 total time=  32.1s
[CV 2/5] END ......C=10, gamma=5, kernel=linear;,

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10, 4], 'gamma': [0.5, 5],
                         'kernel': ('linear', 'rbf')},
             verbose=3)

In [39]:
regressor_svr2.best_params_

{'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}

In [40]:
regressor_svr2.best_score_

0.11008852542985463

In [41]:
ypred4 = regressor_svr2.predict(xtest_scal)
ypred4

array([86.46122308, 87.58954559, 82.66782096, ..., 87.60756485,
       71.95973018, 70.23247436])

In [42]:
np.sqrt(mean_squared_error(ytest, ypred4))

52.62732130725765

In [ ]:
#El error es menor es mejor la estimación